# Preparing data and publishing with Quilt

## Get and unzip primary data table

We're taking data on interstate movement of commodities gathered by the U. S. Census and published at https://www.census.gov/econ/cfs/pums.html.  The relevant parts are in one large zipped CSV, plus the tabs of a modest-sized .xlsx.  We'll begin by grabbing the files.

In [ ]:
import requests
resp = requests.get('https://www.census.gov/econ/cfs/2012/cfs_2012_pumf_csv.zip')
with open('cfs_2012_pumf_csv.zip', 'wb') as outfile:
    outfile.write(resp.content)

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('cfs_2012_pumf_csv.zip', 'r')
zip_ref.extractall()
zip_ref.close()

## Get and unzip data dictionary tables

In [ ]:
url = 'https://www.census.gov/econ/cfs/2012/cfs_2012_pum_file_users_guide_App_A%20(Jun%202015).xlsx'
resp = requests.get(url)
with open('data_dictionary.xlsx', 'wb') as outfile:
    outfile.write(resp.content)

## Generate quilt package

The CSV was straightforward, but most of the tabs of the spreadsheet require some work to get into tidy table form.  We start with doing as much as we can by editing the parameters of `build.yml` - experimenting with arguments to Pandas' [read_excel](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html) to figure out the right arguments to `kwargs`.

In [1]:
!cat build.yml

contents:
  commodity_flow_2012:
    file: cfs_2012_pumf_csv.txt
    transform: csv
    kwargs:
      index_col: 0
  field_descriptions:
    file: data_dictionary.xlsx
    kwargs:
      sheet_name: 0
      skiprows: [0, ]
      usecols: [0, 1]
      skip_footer: 6
  cfs_areas:
    file: data_dictionary.xlsx
    kwargs:
      sheet_name: 1
      skiprows: [0,]
  naics_codes:
    file: data_dictionary.xlsx
    kwargs:
      sheet_name: 2
      skiprows: [0,]
      usecols: 1
      squeeze: True
  sctg_codes:
    file: data_dictionary.xlsx
    kwargs:
      sheet_name: 3
      skiprows: [0,]
      dtype: {'NAICS': str}
  sctg_codes:
    file: data_dictionary.xlsx
    kwargs:
      sheet_name: 3
      skiprows: [0,]
      dtype: {'SCTG': str}
  transport_mode_codes:
    file: data_dictionary.xlsx
    kwargs:
      sheet_name: 4
      skiprows: [0,]
      usecols: 1
      skip_footer: 20
      dtype: {'Mode Code': str}


In [2]:
!quilt build catherinedevlin/us_census_commodity_flow_2012 build.yml

Inferring 'transform: xlsx' for data_dictionary.xlsx
Inferring 'transform: xlsx' for data_dictionary.xlsx
Inferring 'transform: xlsx' for data_dictionary.xlsx
Inferring 'transform: xlsx' for data_dictionary.xlsx
Inferring 'transform: xlsx' for data_dictionary.xlsx
Built catherinedevlin/us_census_commodity_flow_2012 successfully.


## Manual corrections

Some fixes can't be done through `kwargs` loading paramters.  We need to make the changes in Pandas, then save them back to the Quilt package.

In [3]:
from quilt.data.catherinedevlin import us_census_commodity_flow_2012 as cf

In [4]:
field_descriptions = cf.field_descriptions()

In [6]:
field_descriptions

,Field,Description
0,SHIPMT_ID,Shipment identifier
1,ORIG_STATE,FIPS state code of shipment origin
2,ORIG_MA,Metro area of shipment origin
3,ORIG_CFS_AREA,CFS Area of shipment origin
4,DEST_STATE,FIPS state code of shipment destination
5,DEST_MA,Metro area of shipment destination
6,DEST_CFS_AREA,CFS Area of shipment destination
7,NAICS,Industry classification of shipper
8,QUARTER,Quarter of 2012 in which the shipment occurred
9,SCTG,2-digit SCTG Commodity Code of the shipment


### drop nan rows

In [7]:
field_descriptions = field_descriptions[field_descriptions['Field'] != 'nan']

In [8]:
field_descriptions

,Field,Description
0,SHIPMT_ID,Shipment identifier
1,ORIG_STATE,FIPS state code of shipment origin
2,ORIG_MA,Metro area of shipment origin
3,ORIG_CFS_AREA,CFS Area of shipment origin
4,DEST_STATE,FIPS state code of shipment destination
5,DEST_MA,Metro area of shipment destination
6,DEST_CFS_AREA,CFS Area of shipment destination
7,NAICS,Industry classification of shipper
8,QUARTER,Quarter of 2012 in which the shipment occurred
9,SCTG,2-digit SCTG Commodity Code of the shipment


### Write fixed field_descriptions into quilt package

[docs](https://docs.quiltdata.com/edit-a-package.html)

In [10]:
cf._set(['field_descriptions'], field_descriptions)

### Fix two-line headers

In [11]:
cfs_areas = cf.cfs_areas()
cfs_areas.head()

,ORIG_MA,ORIG_STATE,ORIG_CFS_AREA,MA,Description
0,DEST_MA,DEST_STATE,DEST_CFS_AREA,Type,nan
1,122,13,13-122,C,"Atlanta-Athens-Clarke County-Sandy Springs, GA..."
2,142,01,01-142,C,"Birmingham-Hoover-Talladega, AL CFS Area"
3,148,25,25-148,C,"Boston-Worcester-Providence, MA-RI-NH-CT CFS ..."
4,148,33,33-148,C,"Boston-Worcester-Providence, MA-RI-NH-CT CFS ..."


In [12]:
cfs_areas.iloc[0]

ORIG_MA                DEST_MA
ORIG_STATE          DEST_STATE
ORIG_CFS_AREA    DEST_CFS_AREA
MA                        Type
Description                nan
Name: 0, dtype: object

In [13]:
first_vals = dict(zip(cfs_areas.dtypes.index, cfs_areas.iloc[0]))
first_vals

{'Description': 'nan',
 'MA': 'Type',
 'ORIG_CFS_AREA': 'DEST_CFS_AREA',
 'ORIG_MA': 'DEST_MA',
 'ORIG_STATE': 'DEST_STATE'}

In [14]:
first_vals.pop('Description', None)  # this one already correct
corrected_headers = {k: '%s %s' % (k, v) for (k, v) in first_vals.items()}
corrected_headers

{'MA': 'MA Type',
 'ORIG_CFS_AREA': 'ORIG_CFS_AREA DEST_CFS_AREA',
 'ORIG_MA': 'ORIG_MA DEST_MA',
 'ORIG_STATE': 'ORIG_STATE DEST_STATE'}

In [15]:
cfs_areas = cfs_areas.rename(columns=corrected_headers)
cfs_areas.head()

,ORIG_MA DEST_MA,ORIG_STATE DEST_STATE,ORIG_CFS_AREA DEST_CFS_AREA,MA Type,Description
0,DEST_MA,DEST_STATE,DEST_CFS_AREA,Type,nan
1,122,13,13-122,C,"Atlanta-Athens-Clarke County-Sandy Springs, GA..."
2,142,01,01-142,C,"Birmingham-Hoover-Talladega, AL CFS Area"
3,148,25,25-148,C,"Boston-Worcester-Providence, MA-RI-NH-CT CFS ..."
4,148,33,33-148,C,"Boston-Worcester-Providence, MA-RI-NH-CT CFS ..."


In [16]:
cfs_areas = cfs_areas.drop([0]).head()
cfs_areas

,ORIG_MA DEST_MA,ORIG_STATE DEST_STATE,ORIG_CFS_AREA DEST_CFS_AREA,MA Type,Description
1,122,13,13-122,C,"Atlanta-Athens-Clarke County-Sandy Springs, GA..."
2,142,01,01-142,C,"Birmingham-Hoover-Talladega, AL CFS Area"
3,148,25,25-148,C,"Boston-Worcester-Providence, MA-RI-NH-CT CFS ..."
4,148,33,33-148,C,"Boston-Worcester-Providence, MA-RI-NH-CT CFS ..."
5,148,44,44-148,C,"Boston-Worcester-Providence, MA-RI-NH-CT CFS ..."


In [17]:
cf._set(['cfs_areas'], cfs_areas)

## change 'nan' into real NaN

In [19]:
sctg_codes = cf.sctg_codes()
sctg_codes.head()

,SCTG,Description,SCTG Group
0,01,Animals and Fish (live),01-05
1,02,Cereal Grains (includes seed),nan
2,03,"Agricultural Products (excludes Animal Feed, C...",nan
3,04,"Animal Feed, Eggs, Honey, and Other Products o...",nan
4,05,"Meat, Poultry, Fish, Seafood, and Their Prepar...",nan


In [20]:
import numpy as np
sctg_codes = sctg_codes.replace('nan', np.nan)
sctg_codes.head()

,SCTG,Description,SCTG Group
0,01,Animals and Fish (live),01-05
1,02,Cereal Grains (includes seed),NaN
2,03,"Agricultural Products (excludes Animal Feed, C...",NaN
3,04,"Animal Feed, Eggs, Honey, and Other Products o...",NaN
4,05,"Meat, Poultry, Fish, Seafood, and Their Prepar...",NaN


In [21]:
cf._set(['sctg_codes'], sctg_codes)

### Write manual changes to Quilt package

In [22]:
import quilt
quilt.build('catherinedevlin/us_census_commodity_flow_2012', cf)